In [80]:
# ==========================
# 0. IMPORTI I SEED-OVI
# ==========================
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf
from tensorflow.keras import layers, Model

np.random.seed(42)
tf.random.set_seed(42)

# ==========================
# 1. UČITAVANJE I ČIŠĆENJE PODATAKA
# ==========================

df = pd.read_csv("./car-details-v3.csv")

df = df[
    [
        "name",
        "year",
        "selling_price",
        "km_driven",
        "fuel",
        "seller_type",
        "transmission",
        "owner",
        "mileage",
        "engine",
        "max_power",
        "torque",
        "seats",
    ]
]

# Parsiranje numeričkih vrednosti iz string kolona
df["mileage"] = df["mileage"].astype(str).str.extract(r"(\d+\.?\d*)")[0].astype(float)
df["engine"] = df["engine"].astype(str).str.extract(r"(\d+\.?\d*)")[0].astype(float)
df["max_power"] = df["max_power"].astype(str).str.extract(r"(\d+\.?\d*)")[0].astype(float)
df["torque"] = df["torque"].astype(str).str.extract(r"(\d+\.?\d*)")[0].astype(float)
df["seats"] = df["seats"].astype(float)

df["body_coupe"] = df["name"].str.contains(
    "Coupe|Sports|GT|Roadster|Convertible|Cabrio|TT|Z4|S2000|Mustang",
    case=False
).astype(int)

df["body_sedan"] = df["name"].str.contains(
    "Sedan|Dzire|City|Verna|Civic|Corolla|Passat|Octavia|Jetta|C-Class|S-Class",
    case=False
).astype(int)

df["body_suv"] = df["name"].str.contains(
    "Scorpio|Bolero|Fortuner|Safari|Innova|Jeep|XUV|Endeavour|Creta|Harrier|Hector",
    case=False
).astype(int)

df["is_premium_brand"] = df["name"].str.contains(
    "Mercedes|BMW|Audi|Lexus|Jaguar|Volvo|Porsche|Land Rover",
    case=False
).astype(int)

df.dropna(inplace=True)

# ==========================
# 2. NORMALIZACIJA + KATEGORIJE
# ==========================

NUM_COLS = [
    "year",
    "selling_price",
    "km_driven",
    "mileage",
    "engine",
    "max_power",
    "torque",
    "seats",
]

CAT_COLS = ["fuel", "seller_type", "transmission", "owner"]

scaler = MinMaxScaler()
df[NUM_COLS] = scaler.fit_transform(df[NUM_COLS])

for col in CAT_COLS:
    df[col] = df[col].astype("category")

fuel_cat = df["fuel"].cat.categories
seller_cat = df["seller_type"].cat.categories
trans_cat = df["transmission"].cat.categories
owner_cat = df["owner"].cat.categories

df["fuel"] = df["fuel"].cat.codes
df["seller_type"] = df["seller_type"].cat.codes
df["transmission"] = df["transmission"].cat.codes
df["owner"] = df["owner"].cat.codes

# Binarne oznake – koriste se samo u scoring funkciji, ne u modelu
df["is_suv"] = df["name"].str.contains(
    "Scorpio|Bolero|Fortuner|Safari|Sumo|Innova|Jeep|4X4|4WD|Endeavour",
    case=False,
).astype(int)

df["is_sport_model"] = df["name"].str.contains(
    "GTI|GT TSI|TSI|TFSI|vRS|RS\\b|iVTEC|VTEC|Type R|Sports|1.6S|Abarth|Turbo|N Line|"
    "Cooper S|ST Line|AMG|M\\b|M3|M4|M5",
    case=False,
).astype(int)

BIN_COLS = ["is_suv", "is_sport_model"]

num_fuel = len(fuel_cat)
num_seller = len(seller_cat)
num_trans = len(trans_cat)
num_owner = len(owner_cat)

print("fuel:", list(fuel_cat))
print("seller_type:", list(seller_cat))
print("transmission:", list(trans_cat))
print("owner:", list(owner_cat))


def get_code(categories, name, default=0):
    return int(np.where(categories == name)[0][0]) if name in categories else default


petrol_code = get_code(fuel_cat, "Petrol", default=0)
diesel_code = get_code(fuel_cat, "Diesel", default=0)

individual_code = 0
dealer_code = 0
for i, c in enumerate(seller_cat):
    if "Individual" in c:
        individual_code = i
    if "Dealer" in c:
        dealer_code = i

manual_code = get_code(trans_cat, "Manual", default=0)
auto_code = get_code(trans_cat, "Automatic", default=manual_code)

first_owner_code = 0
for i, c in enumerate(owner_cat):
    if "First Owner" in c:
        first_owner_code = i
        break

# Ovo je skup feature-a koji koristimo za automobile (za scoring i model)
df_items = df[NUM_COLS + CAT_COLS + BIN_COLS].copy()

# ==========================
# 3. GENERISANJE USERS PO SEGMENTIMA
# ==========================

def generate_segment_users(n_per_segment=40):
    """
    Generiše sintetičke korisnike sa poljem 'segment' (1..6).
    Ti korisnici se koriste samo za generisanje trening parova.
    """
    users = []

    for _ in range(n_per_segment):
        # 1) Budget Buyer
        users.append(
            {
                "year": np.random.uniform(0.25, 0.55),
                "selling_price": np.random.uniform(0.1, 0.35),
                "km_driven": np.random.uniform(0.3, 0.8),
                "mileage": np.random.uniform(0.6, 1.0),
                "engine": np.random.uniform(0.2, 0.5),
                "max_power": np.random.uniform(0.2, 0.5),
                "torque": np.random.uniform(0.2, 0.5),
                "seats": np.random.uniform(0.3, 0.7),
                "fuel": petrol_code,
                "seller_type": individual_code,
                "transmission": manual_code,
                "owner": first_owner_code,
                "segment": 1,
            }
        )

        # 2) Diesel Commuter
        users.append(
            {
                "year": np.random.uniform(0.45, 0.8),
                "selling_price": np.random.uniform(0.3, 0.6),
                "km_driven": np.random.uniform(0.2, 0.6),
                "mileage": np.random.uniform(0.5, 0.9),
                "engine": np.random.uniform(0.4, 0.7),
                "max_power": np.random.uniform(0.3, 0.6),
                "torque": np.random.uniform(0.4, 0.8),
                "seats": np.random.uniform(0.4, 0.7),
                "fuel": diesel_code,
                "seller_type": dealer_code,
                "transmission": manual_code,
                "owner": first_owner_code,
                "segment": 2,
            }
        )

        # 3) Family Buyer
        users.append(
            {
                "year": np.random.uniform(0.6, 0.9),
                "selling_price": np.random.uniform(0.4, 0.7),
                "km_driven": np.random.uniform(0.1, 0.4),
                "mileage": np.random.uniform(0.4, 0.8),
                "engine": np.random.uniform(0.4, 0.7),
                "max_power": np.random.uniform(0.4, 0.7),
                "torque": np.random.uniform(0.4, 0.7),
                "seats": np.random.uniform(0.6, 1.0),
                "fuel": np.random.choice([petrol_code, diesel_code]),
                "seller_type": dealer_code,
                "transmission": auto_code,
                "owner": first_owner_code,
                "segment": 3,
            }
        )

        # 4) Sport Enthusiast
        users.append(
            {
                "year": np.random.uniform(0.5, 0.9),
                "selling_price": np.random.uniform(0.6, 0.9),
                "km_driven": np.random.uniform(0.05, 0.3),
                "mileage": np.random.uniform(0.3, 0.7),
                "engine": np.random.uniform(0.6, 0.95),
                "max_power": np.random.uniform(0.7, 1.0),
                "torque": np.random.uniform(0.6, 1.0),
                "seats": np.random.uniform(0.3, 0.6),
                "fuel": petrol_code,
                "seller_type": np.random.choice([individual_code, dealer_code]),
                "transmission": manual_code,
                "owner": first_owner_code,
                "segment": 4,
            }
        )

        # 5) Off-road Utility
        users.append(
            {
                "year": np.random.uniform(0.3, 0.8),
                "selling_price": np.random.uniform(0.4, 0.8),
                "km_driven": np.random.uniform(0.3, 0.8),
                "mileage": np.random.uniform(0.3, 0.7),
                "engine": np.random.uniform(0.6, 1.0),
                "max_power": np.random.uniform(0.6, 0.9),
                "torque": np.random.uniform(0.7, 1.0),
                "seats": np.random.uniform(0.6, 1.0),
                "fuel": diesel_code,
                "seller_type": np.random.choice([individual_code, dealer_code]),
                "transmission": manual_code,
                "owner": first_owner_code,
                "segment": 5,
            }
        )

        # 6) Premium / Luxury Urban
        users.append(
            {
                "year": np.random.uniform(0.8, 1.0),
                "selling_price": np.random.uniform(0.7, 1.0),
                "km_driven": np.random.uniform(0.0, 0.3),
                "mileage": np.random.uniform(0.3, 0.7),
                "engine": np.random.uniform(0.6, 0.9),
                "max_power": np.random.uniform(0.6, 0.9),
                "torque": np.random.uniform(0.5, 0.85),
                "seats": np.random.uniform(0.5, 0.9),
                "fuel": petrol_code,
                "seller_type": dealer_code,
                "transmission": auto_code,
                "owner": first_owner_code,
                "segment": 6,
            }
        )

    return pd.DataFrame(users)


users_df = generate_segment_users(40)
print("Users shape:", users_df.shape)

# ==========================
# 4. SEGMENT-BASED SCORING FUNKCIJA
# ==========================

def score_items_for_segment(user_row, cars_df: pd.DataFrame):
    """
    Čisto pravilo-bazirani scoring po segmentima.
    Ne koristi model – samo ručna pravila. Rezultat: np.array score-ova po autu.
    """

    seg = int(user_row["segment"])

    cars = cars_df  # alias
    n = len(cars)

    # bazna numerička sličnost za sve
    user_num = user_row[NUM_COLS].values.astype("float32")
    car_num = cars[NUM_COLS].values.astype("float32")
    diff = np.abs(car_num - user_num)
    base_sim = 1.0 - diff
    base_sim = np.clip(base_sim, 0.0, 1.0)
    base_score = base_sim.sum(axis=1)

    fuel = cars["fuel"].values
    seller = cars["seller_type"].values
    trans = cars["transmission"].values
    owner = cars["owner"].values
    is_suv = cars["is_suv"].values
    is_sport = cars["is_sport_model"].values

    year = cars["year"].values
    price = cars["selling_price"].values
    km = cars["km_driven"].values
    mileage = cars["mileage"].values
    engine = cars["engine"].values
    power = cars["max_power"].values
    torque = cars["torque"].values
    seats = cars["seats"].values

    score = np.zeros(n, dtype="float32")

    # ---------- SEGMENT 1: BUDGET ----------
    if seg == 1:
        score += 2.5 * (1 - np.abs(price - user_row["selling_price"]))
        score += 1.0 * (1 - np.abs(year - user_row["year"]))
        score += 1.5 * (1 - np.abs(km - user_row["km_driven"]))
        score += 2.0 * (1 - np.abs(mileage - user_row["mileage"]))
        score += 1.0 * (fuel == user_row["fuel"])
        score += 1.0 * (seller == user_row["seller_type"])
        score += 0.5 * (owner == user_row["owner"])
        score += base_score

    # ---------- SEGMENT 2: DIESEL COMMUTER ----------
    elif seg == 2:
        score += 3.0 * (fuel == diesel_code)
        score += 2.0 * (1 - np.abs(mileage - user_row["mileage"]))
        score += 1.5 * (1 - np.abs(km - user_row["km_driven"]))
        score += 1.5 * (1 - np.abs(price - user_row["selling_price"]))
        score -= 1.5 * is_suv
        score += 1.0 * (trans == manual_code)
        score += base_score

    # ---------- SEGMENT 3: FAMILY BUYER ----------
    elif seg == 3:
        score += 2.0 * (seats >= 0.6).astype("float32")   # 5+ sedišta
        score += 1.5 * (1 - np.abs(km - user_row["km_driven"]))
        score += 1.5 * (1 - np.abs(year - user_row["year"]))
        score += 1.0 * (trans == auto_code)
        score += 1.0 * (seller == dealer_code)
        score += 0.5 * is_suv  # SUV-ovi blagi plus
        score += base_score

    # ---------- SEGMENT 4: SPORT ENTHUSIAST ----------
    elif seg == 4:
        score += 4.0 * power
        score += 3.0 * engine
        score += 2.0 * torque
        score += 2.0 * is_sport
        score -= 3.0 * is_suv
        score += 2.0 * (seats <= 0.5).astype("float32")
        score += 1.5 * (fuel == petrol_code)
        score += 1.0 * (trans == manual_code)
        score += 10 * df["body_coupe"]
        score -= 8 * df["body_suv"]
        score -= 5 * df["body_sedan"]
        score += 15 * df["is_sport_model"]
        score += base_score

    # ---------- SEGMENT 5: OFF-ROAD ----------
    elif seg == 5:
        score += 4.0 * is_suv
        score += 3.0 * torque
        score += 2.0 * engine
        score += 1.5 * (fuel == diesel_code)
        score += 1.0 * (seats >= 0.6).astype("float32")
        score += 1.0 * (trans == manual_code)
        score += base_score

    # ---------- SEGMENT 6: LUXURY / PREMIUM ----------
    elif seg == 6:
        score += 3.0 * (1 - np.abs(price - user_row["selling_price"]))
        score += 2.5 * (1 - np.abs(year - user_row["year"]))
        score += 2.0 * (trans == auto_code)
        score += 1.5 * (seller == dealer_code)
        score += 2.0 * (seats >= 0.6).astype("float32")
        score += 1.5 * (fuel == petrol_code)
        score += 1.0 * power
        score += 8 * df["body_sedan"]
        score += 10 * df["is_premium_brand"]
        score -= 6 * df["body_suv"] 
        score -= 20 * (df["is_premium_brand"] == 0)
        score += base_score

    else:
        score += base_score

    return score


# ==========================
# 5. GENERISANJE TRAINING PAROVA (bez segment kao feature)
# ==========================

def generate_training_pairs_fast(users_df, cars_df, n_pos=15, n_neg=15):
    u_num_list = []
    u_fuel_list = []
    u_seller_list = []
    u_trans_list = []
    u_owner_list = []

    i_num_list = []
    i_fuel_list = []
    i_seller_list = []
    i_trans_list = []
    i_owner_list = []

    y_list = []

    for _, user in users_df.iterrows():
        scores = score_items_for_segment(user, cars_df)
        idx_sorted = np.argsort(scores)
        pos_idx = idx_sorted[-n_pos:]
        neg_idx = idx_sorted[:n_neg]

        def add_pairs(indices, label):
            for idx in indices:
                car = cars_df.iloc[idx]

                u_num_list.append(user[NUM_COLS].values.astype("float32"))
                u_fuel_list.append(int(user["fuel"]))
                u_seller_list.append(int(user["seller_type"]))
                u_trans_list.append(int(user["transmission"]))
                u_owner_list.append(int(user["owner"]))

                i_num_list.append(car[NUM_COLS].values.astype("float32"))
                i_fuel_list.append(int(car["fuel"]))
                i_seller_list.append(int(car["seller_type"]))
                i_trans_list.append(int(car["transmission"]))
                i_owner_list.append(int(car["owner"]))

                y_list.append(float(label))

        add_pairs(pos_idx, 1.0)
        add_pairs(neg_idx, 0.0)

    u_num = np.stack(u_num_list).astype("float32")
    u_fuel = np.array(u_fuel_list, dtype="int32")
    u_seller = np.array(u_seller_list, dtype="int32")
    u_trans = np.array(u_trans_list, dtype="int32")
    u_owner = np.array(u_owner_list, dtype="int32")

    i_num = np.stack(i_num_list).astype("float32")
    i_fuel = np.array(i_fuel_list, dtype="int32")
    i_seller = np.array(i_seller_list, dtype="int32")
    i_trans = np.array(i_trans_list, dtype="int32")
    i_owner = np.array(i_owner_list, dtype="int32")

    y = np.array(y_list, dtype="float32")

    return (
        u_num,
        u_fuel,
        u_seller,
        u_trans,
        u_owner,
        i_num,
        i_fuel,
        i_seller,
        i_trans,
        i_owner,
        y,
    )


(
    u_num,
    u_fuel,
    u_seller,
    u_trans,
    u_owner,
    i_num,
    i_fuel,
    i_seller,
    i_trans,
    i_owner,
    y,
) = generate_training_pairs_fast(users_df, df_items, n_pos=15, n_neg=15)

print("u_num:", u_num.shape)
print("i_num:", i_num.shape)
print("y:", y.shape)

# ==========================
# 6. TWO–TOWER MODEL
# ==========================

embedding_dim = 32
num_numeric = len(NUM_COLS)

# USER tower
user_numeric_in = layers.Input(shape=(num_numeric,), name="user_num")
user_fuel_in = layers.Input(shape=(), dtype="int32", name="user_fuel")
user_seller_in = layers.Input(shape=(), dtype="int32", name="user_seller")
user_trans_in = layers.Input(shape=(), dtype="int32", name="user_trans")
user_owner_in = layers.Input(shape=(), dtype="int32", name="user_owner")

uf_emb = layers.Embedding(num_fuel, 8)(user_fuel_in)
us_emb = layers.Embedding(num_seller, 8)(user_seller_in)
ut_emb = layers.Embedding(num_trans, 8)(user_trans_in)
uo_emb = layers.Embedding(num_owner, 8)(user_owner_in)

u_concat = layers.Concatenate()(
    [
        user_numeric_in,
        layers.Flatten()(uf_emb),
        layers.Flatten()(us_emb),
        layers.Flatten()(ut_emb),
        layers.Flatten()(uo_emb),
    ]
)

u_hidden = layers.Dense(128, activation="relu")(u_concat)
u_hidden = layers.Dropout(0.2)(u_hidden)
u_hidden = layers.Dense(64, activation="relu")(u_hidden)
u_vec = layers.Dense(embedding_dim)(u_hidden)

user_tower = Model(
    inputs=[user_numeric_in, user_fuel_in, user_seller_in, user_trans_in, user_owner_in],
    outputs=u_vec,
)

# ITEM tower
item_numeric_in = layers.Input(shape=(num_numeric,), name="item_num")
item_fuel_in = layers.Input(shape=(), dtype="int32", name="item_fuel")
item_seller_in = layers.Input(shape=(), dtype="int32", name="item_seller")
item_trans_in = layers.Input(shape=(), dtype="int32", name="item_trans")
item_owner_in = layers.Input(shape=(), dtype="int32", name="item_owner")

if_emb = layers.Embedding(num_fuel, 8)(item_fuel_in)
is_emb = layers.Embedding(num_seller, 8)(item_seller_in)
it_emb = layers.Embedding(num_trans, 8)(item_trans_in)
io_emb = layers.Embedding(num_owner, 8)(item_owner_in)

i_concat = layers.Concatenate()(
    [
        item_numeric_in,
        layers.Flatten()(if_emb),
        layers.Flatten()(is_emb),
        layers.Flatten()(it_emb),
        layers.Flatten()(io_emb),
    ]
)

i_hidden = layers.Dense(128, activation="relu")(i_concat)
i_hidden = layers.Dropout(0.2)(i_hidden)
i_hidden = layers.Dense(64, activation="relu")(i_hidden)
i_vec = layers.Dense(embedding_dim)(i_hidden)

item_tower = Model(
    inputs=[item_numeric_in, item_fuel_in, item_seller_in, item_trans_in, item_owner_in],
    outputs=i_vec,
)

dot_score = layers.Dot(axes=1)([u_vec, i_vec])

model = Model(
    inputs=[
        user_numeric_in,
        user_fuel_in,
        user_seller_in,
        user_trans_in,
        user_owner_in,
        item_numeric_in,
        item_fuel_in,
        item_seller_in,
        item_trans_in,
        item_owner_in,
    ],
    outputs=dot_score,
)

model.compile(optimizer="adam", loss="binary_crossentropy")
model.summary()

# ==========================
# 7. TRENING
# ==========================

history = model.fit(
    [u_num, u_fuel, u_seller, u_trans, u_owner, i_num, i_fuel, i_seller, i_trans, i_owner],
    y,
    epochs=10,
    batch_size=64,
    verbose=1,
)

# ==========================
# 8. PREKOMPJUTACIJA ITEM EMBEDDINGA
# ==========================

def build_item_inputs_from_df(cars_df: pd.DataFrame):
    num = cars_df[NUM_COLS].values.astype("float32")
    fuel = cars_df["fuel"].values.astype("int32")
    seller = cars_df["seller_type"].values.astype("int32")
    trans = cars_df["transmission"].values.astype("int32")
    owner = cars_df["owner"].values.astype("int32")
    return num, fuel, seller, trans, owner


item_num_all, item_fuel_all, item_seller_all, item_trans_all, item_owner_all = build_item_inputs_from_df(df_items)
item_embeddings = item_tower.predict(
    [item_num_all, item_fuel_all, item_seller_all, item_trans_all, item_owner_all],
    verbose=0,
)

# ==========================
# 9. FUNKCIJA ZA PREPORUKE ZA BILO KOG USERA
# ==========================

def recommend_for_user(user_pref: dict, top_n=10):
    """
    user_pref: dict sa ključevima NUM_COLS + CAT_COLS
    (NUM_COLS već u [0,1]; CAT_COLS su int kodovi).
    """
    user_num = np.array([[user_pref[c] for c in NUM_COLS]], dtype="float32")
    user_fuel = np.array([user_pref["fuel"]], dtype="int32")
    user_seller = np.array([user_pref["seller_type"]], dtype="int32")
    user_trans = np.array([user_pref["transmission"]], dtype="int32")
    user_owner = np.array([user_pref["owner"]], dtype="int32")

    u_emb = user_tower.predict(
        [user_num, user_fuel, user_seller, user_trans, user_owner],
        verbose=0,
    )

    scores = cosine_similarity(u_emb, item_embeddings)[0]
    top_idx = np.argsort(scores)[::-1][:top_n]

    return df.iloc[top_idx][
        ["name", "year", "selling_price", "km_driven", "fuel", "transmission", "owner"]
    ]

# ==========================
# 10. TEST KORISNICI (SEGMENT-LIKE)
# ==========================

sport_user = {
    "year": 0.6,
    "selling_price": 0.8,
    "km_driven": 0.2,
    "mileage": 0.4,
    "engine": 0.85,
    "max_power": 0.9,
    "torque": 0.85,
    "seats": 0.45,
    "fuel": petrol_code,
    "seller_type": dealer_code,
    "transmission": manual_code,
    "owner": first_owner_code,
}

luxury_user = {
    "year": 0.9,
    "selling_price": 0.9,
    "km_driven": 0.15,
    "mileage": 0.5,
    "engine": 0.8,
    "max_power": 0.8,
    "torque": 0.75,
    "seats": 0.8,
    "fuel": petrol_code,
    "seller_type": dealer_code,
    "transmission": auto_code,
    "owner": first_owner_code,
}

family_user = {
    "year": 0.75,
    "selling_price": 0.6,
    "km_driven": 0.25,
    "mileage": 0.6,
    "engine": 0.6,
    "max_power": 0.6,
    "torque": 0.6,
    "seats": 0.8,
    "fuel": diesel_code,
    "seller_type": dealer_code,
    "transmission": auto_code,
    "owner": first_owner_code,
}

budget_user = {
    "year": 0.4,
    "selling_price": 0.2,
    "km_driven": 0.5,
    "mileage": 0.8,
    "engine": 0.4,
    "max_power": 0.4,
    "torque": 0.4,
    "seats": 0.5,
    "fuel": petrol_code,
    "seller_type": individual_code,
    "transmission": manual_code,
    "owner": first_owner_code,
}

offroad_user = {
    "year": 0.6,
    "selling_price": 0.6,
    "km_driven": 0.5,
    "mileage": 0.5,
    "engine": 0.9,
    "max_power": 0.8,
    "torque": 0.9,
    "seats": 0.8,
    "fuel": diesel_code,
    "seller_type": dealer_code,
    "transmission": manual_code,
    "owner": first_owner_code,
}

diesel_commuter_user = {
    "year": 0.7,
    "selling_price": 0.5,
    "km_driven": 0.4,
    "mileage": 0.8,
    "engine": 0.6,
    "max_power": 0.5,
    "torque": 0.7,
    "seats": 0.6,
    "fuel": diesel_code,
    "seller_type": dealer_code,
    "transmission": manual_code,
    "owner": first_owner_code,
}

print("==================== SPORT USER ====================")
print(recommend_for_user(sport_user, top_n=10))

print("\n==================== LUXURY USER ====================")
print(recommend_for_user(luxury_user, top_n=10))

print("\n==================== FAMILY USER ====================")
print(recommend_for_user(family_user, top_n=10))

print("\n==================== BUDGET USER ====================")
print(recommend_for_user(budget_user, top_n=10))

print("\n==================== OFFROAD USER ====================")
print(recommend_for_user(offroad_user, top_n=10))

print("\n==================== DIESEL COMMUTER USER ====================")
print(recommend_for_user(diesel_commuter_user, top_n=10))


fuel: ['CNG', 'Diesel', 'LPG', 'Petrol']
seller_type: ['Dealer', 'Individual', 'Trustmark Dealer']
transmission: ['Automatic', 'Manual']
owner: ['First Owner', 'Fourth & Above Owner', 'Second Owner', 'Test Drive Car', 'Third Owner']
Users shape: (240, 13)
u_num: (7200, 8)
i_num: (7200, 8)
y: (7200,)


Model: "functional_43"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_fuel           │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_seller         │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_trans          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_owner          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_fuel           │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_seller         │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_trans          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_owner          │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_118       │ (None, 8)         │         32 │ user_fuel[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_119       │ (None, 8)         │         24 │ user_seller[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_120       │ (None, 8)         │         16 │ user_trans[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_121       │ (None, 8)         │         40 │ user_owner[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_122       │ (None, 8)         │         32 │ item_fuel[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_123       │ (None, 8)         │         24 │ item_seller[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_124       │ (None, 8)         │         16 │ item_trans[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_125       │ (None, 8)         │         40 │ item_owner[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ user_num            │ (None, 8)         │          0 │ -               

 Total params: 31,392 (122.62 KB)

 Trainable params: 31,392 (122.62 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.6696
Epoch 2/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.2884
Epoch 3/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.1458
Epoch 4/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.1112
Epoch 5/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.2125
Epoch 6/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0405
Epoch 7/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0229
Epoch 8/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0245
Epoch 9/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 0.0257
Epoch 10/10
113/113 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - loss: 0.0141
==================== SPORT USER ====================
                                   name      year  selling_price  km_driven  \
170       Volvo XC90 T8 Excellence BSIV  0.884615       1.000000   0.012709   
1063         Toyota Corolla Altis 1.8 G  0.923077       0.137412   0.005733   
4089         Toyota Corolla A